# 공공데이터포털 API 활용

In [ ]:
import requests, xmltodict
import pandas as pd

# 중첩된 딕셔너리 편하게 조회
from pprint import pprint

# 반복문에서의 진행도 확인
from tqdm.notebook import tqdm

In [ ]:
# 요청 url 및 api카
URL = 'https://apis.data.go.kr/1613000/RTMSDataSvcAptTrade/getRTMSDataSvcAptTrade'
API_KEY = 'a46bbaf106e41963c3883db630366f91e4960f1a08c831475517864811d806f4'

In [ ]:
# 요청 변수
query = {
    'LAWD_CD': '11680',
    'DEAL_YMD': '202512',
    'serviceKey': API_KEY,
    'numOfRows': 9999,
    'pageNo': 1
}

In [ ]:
# HTTP Request(요청)
# res를 HTTP Respomse 객체라고 함
res = requests.get(url=URL, params=query)

In [ ]:
# 응답 상태 코드 확인(200이면 정상)
res.status_code # 200

In [ ]:
# 응답 헤더 중 콘텐츠 타입 확인
res.headers

In [ ]:
# 데이터 타입과 인코딩 확인
res.headers['content-type']
# 'application/xml;charset=utf-8'

In [ ]:
# 응답 바디 문자열 확인
res.text

In [ ]:
# xml 형태의 문자열을 딕셔너리로 변환
data = xmltodict.parse(xml_input=res.text)

In [ ]:
pprint(data)

In [ ]:
# 데이터프레임으로 변환
df = pd.DataFrame(data['response']['body']['items']['item'])

In [ ]:
df.info()

In [ ]:
def apt_price(areaCd, ymonth, nrow=9999, page=1):
    # url 주소 및 api 키
    URL = 'https://apis.data.go.kr/1613000/RTMSDataSvcAptTrade/getRTMSDataSvcAptTrade'
    API_KEY = 'a46bbaf106e41963c3883db630366f91e4960f1a08c831475517864811d806f4'
    # 요청 변수
    query = {
        'LAWD_CD': areaCd,
        'DEAL_YMD': ymonth,
        'serviceKey': API_KEY,
        'numOfRows': nrow,
        'pageNo': page
    }
    # requests 가져오기
    res = requests.get(url=URL, params=query)
    # xml 데이터 추출
    data = xmltodict.parse(xml_input=res.text)
    # 데이터프레임으로 변환
    df = pd.DataFrame(data['response']['body']['items']['item'])
    
    return df

In [ ]:
df = apt_price(areaCd='11680', ymonth='202512')
df.shape

In [ ]:
ymonths = [str(i) for i in range(202501, 202513)]
print(ymonths)

In [ ]:
# ymonths 기간의 데이터를 모두 담을 빈 데이터프레임 생성
dfs = pd.DataFrame()

for ymonth in tqdm(ymonths):
    # ymonth를 순회하면서 df 생성
    df = apt_price(areaCd='11680', ymonth=ymonth)
    # 생성 된 df를 dfs에 병합
    dfs = pd.concat([dfs, df], ignore_index=True)

In [ ]:
dfs.shape

In [ ]:
dfs.head()

In [ ]:
dfs.columns

In [ ]:
cols = ['aptDong', 'aptNm', 'buildYear', 'dealAmount', 
        'dealDay', 'dealMonth', 'dealYear', 'excluUseAr',
        'floor', 'jibun', 'rgstDate', 'sggCd', 'umdNm']
dfs = dfs.loc[:, cols]
dfs.head()

In [ ]:
dfs.columns = ['동', '단지명', '입주년도', '거래금액', 
            '계약일', '계약월', '계약연도', '전용면적',
            '층', '지번', '등기일자', '지역코드', '법정동']

dfs.head()

In [ ]:
dfs['시도명'] = '서울특별시'
dfs['시군구'] = '강남구'
dfs.head()

In [ ]:
dfs = dfs.drop(columns=['동', '지역코드'])
dfs.head()

In [ ]:
# 전체 평균
# 문자열 처리 replace(, -> '')
dfs['거래금액'] = dfs['거래금액'].str.replace(pat=',', repl='')

# object 타입을 int로 변환
dfs['거래금액'] = dfs['거래금액'].astype(int)

dfs['거래금액'].mean()

In [ ]:
# 특정 컬럼 기준으로 평균 계산
dfs.groupby(by='법정동')['거래금액'].mean().sort_values()

In [ ]:
# 내림차순 정렬
dfs.groupby(by='법정동')['거래금액'].mean().sort_values(ascending=False)

### 실습

In [ ]:
ymonths = [str(i) for i in range(202501, 202513)]
print(ymonths)

In [ ]:
# 빈 데이터프레임 생성
dfs = pd.DataFrame()
areaCd = '26350'

# 25년도 1월부터 12월까지 순회하는 반복문 생성
for ymonth in ymonths:
    # apt_price 함수를 사용해 데이터프레임 생성
    df = apt_price(areaCd=areaCd, ymonth=ymonth)
    dfs = pd.concat([dfs, df], ignore_index=True)

In [ ]:
# 생성된 데이터프레임 조회
dfs.info()
dfs.head()

In [ ]:
# 컬럼 조회
dfs.columns

In [ ]:
# 필요한 컬럼만 남기고 삭제
cols = ['aptDong', 'aptNm', 'buildYear', 'dealAmount', 
        'dealDay', 'dealMonth', 'dealYear', 'excluUseAr',
        'floor', 'jibun', 'rgstDate', 'sggCd', 'umdNm']

# 팬시 인덱싱으로 필요한 컬럼만 재할당
dfs = dfs.loc[:, cols]

dfs.head()

In [ ]:
# 컬럼명 한글로 변경
dfs.columns = ['동', '단지명', '입주년도', '거래금액', 
            '계약일', '계약월', '계약연도', '전용면적',
            '층', '지번', '등기일자', '지역코드', '법정동']

dfs.head()

In [ ]:
# 동, 지역코드 컬럼 삭제
dfs = dfs.drop(columns=['동', '지역코드'])

dfs.head()

In [ ]:
# 데이터 타입 확인
dfs.info()

In [ ]:
# 거래금액 object -> int로 변경
dfs['거래금액']

In [ ]:
# str.replace로 , 제거 후 재할당
dfs['거래금액'] = dfs['거래금액'].str.replace(pat=',', repl='')
# , 제거된 object를 int 변환하여 재할당
dfs['거래금액'] = dfs['거래금액'].astype(int)

dfs['거래금액']

In [ ]:
dfs.info()
dfs.head()

In [ ]:
# 거래가격 확인
dfs['거래금액'].mean() # np.float64(64032.96877061799)
dfs['거래금액'].min() # np.int64(3900)
dfs['거래금액'].max() # np.int64(570000)
dfs['거래금액'].median() # np.float64(51000.0)

In [ ]:
# 법정동 컬럼의 고유값 목록 확인
len(dfs['법정동'].unique()) # 7
dfs['법정동'].unique()

In [ ]:
dfs.groupby('법정동')['거래금액'].size()

In [ ]:
dfs['층'].astype(int).max()

In [ ]:
dfs.groupby('계약월').size().sort_values(ascending=False)

In [ ]:
dfs[(dfs['법정동'] == '좌동') & (dfs['거래금액'] >= dfs['거래금액'].mean())].sort_values('거래금액', ascending=False).head(20)